In [18]:
import os # OS e.g directory structure
import numpy as np # linear algebra
import scipy as sc  # scientific computing
import pandas as pd # data processing, file I/O
import seaborn as sns  # visualization
import matplotlib.pyplot as plt # visualization
import warnings
warnings.filterwarnings("ignore")

In [19]:

import sys

from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler,OneHotEncoder,StringIndexer,VectorIndexer

In [20]:

! ls -la
! head -n 3 library-collection-inventory.csv

total 22983832
drwxr-xr-x  5 macbookpro  staff          160 20 Mai 20:40 .
drwxr-xr-x  6 macbookpro  staff          192  6 Mai 14:27 ..
drwxr-xr-x  3 macbookpro  staff           96  7 Mai 00:31 .ipynb_checkpoints
-rw-r--r--@ 1 macbookpro  staff        50232 20 Mai 20:40 TP_BIG_DATA.ipynb
-rw-rw-r--@ 1 macbookpro  staff  11764863851  1 Dez  2019 library-collection-inventory.csv
BibNum,Title,Author,ISBN,PublicationYear,Publisher,Subjects,ItemType,ItemCollection,FloatingItem,ItemLocation,ReportDate,ItemCount
3011076,"A tale of two friends / adapted by Ellie O'Ryan ; illustrated by Tom Caulfield, Frederick Gardner, Megan Petasky, and Allen Tam.","O'Ryan, Ellie","1481425730, 1481425749, 9781481425735, 9781481425742",2014.,"Simon Spotlight,","Musicians Fiction, Bullfighters Fiction, Best friends Fiction, Friendship Fiction, Adventure and adventurers Fiction",jcbk,ncrdr,Floating,qna,2017-09-01T00:00:00.000,1
2248846,"Naruto. Vol. 1, Uzumaki Naruto / story and art by Masashi Kishimoto ; [Engli

In [21]:
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, TimestampType, DoubleType,ArrayType

fire_schema = StructType([StructField("BibNum", IntegerType(),True),
                             StructField("Title", StringType(),True),
                             StructField("Author", StringType(),True),
                             StructField("ISBN", IntegerType(),True),
                             StructField("PublicationYear", IntegerType(),True),
                             StructField("Publisher", StringType(),True),
                             StructField("Subjects", StringType(), True),
                             StructField("ItemType", StringType(),True),
                             StructField("ItemCollection", StringType(),True),
                             StructField("FloatingItem", StringType(),True),
                             StructField("ItemLocation", StringType(),True),
                             StructField("ReportDate", TimestampType(),True),
                             StructField("ItemCount", IntegerType(),True)
                            ])


In [22]:


df = spark.read.csv('library-collection-inventory.csv',header=True, schema=fire_schema, sep=",")



### Exploratory data analysis

In [23]:
df.printSchema()
cols = df.columns
df.count()

root
 |-- BibNum: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Author: string (nullable = true)
 |-- ISBN: integer (nullable = true)
 |-- PublicationYear: integer (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- Subjects: string (nullable = true)
 |-- ItemType: string (nullable = true)
 |-- ItemCollection: string (nullable = true)
 |-- FloatingItem: string (nullable = true)
 |-- ItemLocation: string (nullable = true)
 |-- ReportDate: timestamp (nullable = true)
 |-- ItemCount: integer (nullable = true)



35531308

# Data cleaning




In [24]:
df_1 = df.dropna(subset=["Author","Publisher"])

In [25]:
df_1.select("Subjects").filter("PublicationYear is not null").show()

+--------------------+
|            Subjects|
+--------------------+
|Grandfathers Juve...|
|Brigham Young Uni...|
|Cooking Japanese,...|
|Counting Fiction,...|
|Parents Death Fic...|
|Missing persons F...|
|Paper Juvenile li...|
|Mount Rainier Nat...|
|Giraffe Juvenile ...|
|Sharks Juvenile f...|
|Undercover operat...|
|                null|
|Superheroes Comic...|
|                null|
|Probation Fiction...|
|Unemployment Juve...|
|Rand Paul 1914 19...|
|                null|
|Russo Japanese Wa...|
|Quilting, Quiltin...|
+--------------------+
only showing top 20 rows



In [26]:
max_row =10000

df_clean = df_1.limit(max_row)

df_clean.count()

10000

In [ ]:

def transformToIndex(inputArr, data):
    indexer=None 
    for col_in, col_out in inputArr.items():
        
        indexer = StringIndexer(inputCol=col_in, outputCol=col_out)
        data = indexer.fit(data).transform(data)
        
        # converçao para int 
        data = data.withColumn(col_out, data[col_out].cast(IntegerType()))
    return data
col_to_trasform = {
                   'Author':'AuthorIndex', 
                   'Publisher':'PublisherIndex',
                   'ItemCollection': 'ItemCollectionIndex'
                   }
#'Author':'AuthorIndex', 'Publisher':'PublisherIndex','ItemCollection': 'ItemCollectionIndex'

In [ ]:
df_clean = transformToIndex(col_to_trasform,df_clean)

transformToIndex(col_to_trasform,df_clean)

In [ ]:
df_clean.printSchema()


### Tratamento da Variavel Preditora 

In [ ]:
# conversao de string para array


In [11]:
df_clean.printSchema()
#df_indexed = df_clean.select("ItemTypeIndex","AuthorIndex","PublisherIndex","ItemCollectionIndex")

root
 |-- BibNum: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Author: string (nullable = true)
 |-- ISBN: integer (nullable = true)
 |-- PublicationYear: integer (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- Subjects: string (nullable = true)
 |-- ItemType: string (nullable = true)
 |-- ItemCollection: string (nullable = true)
 |-- FloatingItem: string (nullable = true)
 |-- ItemLocation: string (nullable = true)
 |-- ReportDate: timestamp (nullable = true)
 |-- ItemCount: integer (nullable = true)



In [27]:
col_to_drop = ['ItemCount','FloatingItem','ReportDate']

In [28]:
cols_feactures = [c for c in cols if c not in col_to_drop]
col_interesting = ['PublisherIndex','AuthorIndex','SubjectsIndex','ItemCollectionIndex']

In [ ]:
df_clean.write.format("parquet").mode("overwrite")\
                .save("library-file.parquet")

In [ ]:
! ls -la

In [29]:

# Correlation requires vectors so prior we convert to vector column
vector_col = "corr_features"
assembler = VectorAssembler(inputCols=col_interesting, outputCol=vector_col)
df_vector = assembler.transform(df_clean).select(vector_col)

# get correlation matrix

matrix = Correlation.corr(df_vector, vector_col).collect()[0][0]
corrmatrix = matrix.toArray().tolist()
# corrmatrix
corrmatrix

IllegalArgumentException: PublisherIndex does not exist. Available: BibNum, Title, Author, ISBN, PublicationYear, Publisher, Subjects, ItemType, ItemCollection, FloatingItem, ItemLocation, ReportDate, ItemCount

In [ ]:
def ScatterPlot(df, width, height):
    plt.figure(figsize=(width, height))
    plt.pcolor(df)
    plt.yticks(np.arange(0.5, len(df.index), 1), df.index)
    plt.xticks(np.arange(0.5, len(df.columns), 1), df.columns)
    plt.show()

In [ ]:
# Python DataFrame for visualization

df_plot = pd.DataFrame(data=corrmatrix)
# df_plot

In [ ]:
ScatterPlot(df_plot, 20, 20)

In [ ]:
df_clean = df_clean.withColumn('ISBN', df_clean['ISBN'].cast(DoubleType()))

In [30]:

categoricalColumns = ['Title', 'Author','Publisher']
stages = []
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]
label_stringIdx = StringIndexer(inputCol = 'ItemLocation', outputCol = 'label')
stages += [label_stringIdx]
numericCols = ['ISBN', 'ItemCount']
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [31]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(df_clean)
df_clean = pipelineModel.transform(df_clean)
selectedCols = ['label', 'features'] + cols
df_clean = df_clean.select(selectedCols)
df_clean.printSchema()

root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- BibNum: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Author: string (nullable = true)
 |-- ISBN: integer (nullable = true)
 |-- PublicationYear: integer (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- Subjects: string (nullable = true)
 |-- ItemType: string (nullable = true)
 |-- ItemCollection: string (nullable = true)
 |-- FloatingItem: string (nullable = true)
 |-- ItemLocation: string (nullable = true)
 |-- ReportDate: timestamp (nullable = true)
 |-- ItemCount: integer (nullable = true)



In [32]:
train, test = df_clean.randomSplit([0.7, 0.3], seed = 10)


In [33]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter=10)
lrModel = lr.fit(train)

Py4JJavaError: An error occurred while calling o1023.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 31.0 failed 1 times, most recent failure: Lost task 0.0 in stage 31.0 (TID 1336) (192.168.100.89 executor driver): org.apache.spark.SparkException: Failed to execute user defined function(VectorAssembler$$Lambda$3080/734771591: (struct<TitleclassVec:struct<type:tinyint,size:int,indices:array<int>,values:array<double>>,AuthorclassVec:struct<type:tinyint,size:int,indices:array<int>,values:array<double>>,PublisherclassVec:struct<type:tinyint,size:int,indices:array<int>,values:array<double>>,ISBN_double_VectorAssembler_c330e3532e50:double,ItemCount_double_VectorAssembler_c330e3532e50:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage2.sort_addToSorter_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedC


#### Index labels, adding metadata to the label column.
#### Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="Subjects", outputCol="SubjectsIndex").fit(df_clean)
#### Automatically identify categorical features, and index them.
#### We specify maxCategories so features with > 4 distinct values are treated as continuous.

featureIndexer =\
VectorIndexer(inputCol=["Author"], outputCol="AuthorIndex", maxCategories=4).fit(df_clean)

#### Split the data into training and test sets (20% held out for testing)
(trainingData, testData) = data.randomSplit([0.8, 0.2])

#### Train a DecisionTree model.
dt = DecisionTreeClassifier(labelCol="label", featuresCol="AuthorIndex")

### Chain indexers and tree in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, dt])

### Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

### Make predictions.
predictions = model.transform(testData)

### Select example rows to display.
predictions.select("prediction", "indexedLabel", "features").show(5)

### Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g " % (1.0 - accuracy))

treeModel = model.stages[2]
### summary only
print(treeModel)

train, test = df_clean.randomSplit([0.7, 0.3], seed = 10)
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))